# Proyecto Final – Comparativa de LLMs con y sin RAG para generación en Guaraní/Jopará**Diplomado FP-UNA**Este notebook incluye toda la estructura solicitada, lista para ejecución en Colab.

# 🧩 Celda 1 – Título e introducción (Markdown)


# Proyecto Final – Comparativa de LLMs con y sin RAG para generación en Guaraní/Jopará

**Diplomado: PLN e IA – FP-UNA**  

**Integrantes del grupo:**  
- Nombre 1 (C.I. / Legajo)  
- Nombre 2 (C.I. / Legajo)  
- Nombre 3 (C.I. / Legajo)

---

Este notebook implementa y evalúa un agente conversacional para transformar oraciones en guaraní según instrucciones de cambio (`Change`), comparando:

1. **LLMs sin RAG (solo prompting)**  
2. **LLMs con RAG (contexto gramatical y diccionarios en Guaraní/Jopará)**  

Se siguen las indicaciones de la consigna:

- Uso del dataset de AmericasNLP 2025 (tarea ST2, Guaraní).  
- Uso de documentación lingüística (gramática, diccionario, etc.) como base de conocimiento.  
- Comparación de **al menos dos modelos LLM** y de los enfoques **con vs. sin RAG**, discutiendo resultados.


# 🧩 Celda 2 – Instalación de librerías (Código)

In [ ]:
!pip install -q faiss-cpu pypdf sacrebleu openai gdown sentence-transformers tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.3 MB/s eta 0:00:00


# 🧩 Celda 3 – Imports y configuración de OpenRouter (Código)


In [ ]:
import os
import numpy as np
import pandas as pd
import faiss
import sacrebleu

from tqdm.auto import tqdm
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from openai import OpenAI

# ===============================
# Configuración de la API OpenRouter
# ===============================

openrouter_key = os.environ.get("OPENROUTER_API_KEY")

if openrouter_key is None:
    try:
        # Intentar obtener la clave desde los "user secrets" de Colab
        from google.colab import userdata
        openrouter_key = userdata.get("OPENROUTER_API_KEY")
        os.environ["OPENROUTER_API_KEY"] = openrouter_key
    except Exception:
        raise RuntimeError(
            "No se encontró OPENROUTER_API_KEY. "
            "Configúrala en Colab (Settings -> User secrets) "
            "o asigna os.environ['OPENROUTER_API_KEY'] manualmente."
        )

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_key,
)

# ===============================
# Modelos a utilizar (2 LLMs distintos)
# ===============================
# Podés cambiarlos por otros modelos de OpenRouter si lo deseás,
# pero siempre manteniendo al menos dos modelos para la comparación.

LLM_MODEL_A = "meta-llama/llama-3.1-8b-instruct"
LLM_MODEL_B = "mistralai/mistral-small-latest"

print("Cliente OpenRouter configurado correctamente.")
print("Modelo A:", LLM_MODEL_A)
print("Modelo B:", LLM_MODEL_B)


Cliente OpenRouter configurado correctamente.
Modelo A: meta-llama/llama-3.1-8b-instruct
Modelo B: mistralai/mistral-small-latest


# 🧩 Celda 4 – Sección dataset AmericasNLP (Markdown)

## 3. Dataset AmericasNLP 2025 – Tarea ST2 (Guaraní)

En esta sección se descarga y prepara el dataset de AmericasNLP 2025 para la tarea ST2 (Educational Materials, Guaraní).  
Trabajaremos principalmente con los campos:

- `Source`: oración original en guaraní.  
- `Change`: instrucción de modificación (p.ej. `TYPE:AFF`).  
- `Target`: oración resultante esperada.  


# 🧩 Celda 5 – Descarga y carga del dataset (Código)

In [ ]:
import os

BASE_DATA_DIR = "./americasnlp_guarani"
os.makedirs(BASE_DATA_DIR, exist_ok=True)

urls = {
    "train": "https://raw.githubusercontent.com/AmericasNLP/americasnlp2025/main/ST2_EducationalMaterials/data/guarani-train.tsv",
    "dev":   "https://raw.githubusercontent.com/AmericasNLP/americasnlp2025/main/ST2_EducationalMaterials/data/guarani-dev.tsv",
    "test":  "https://raw.githubusercontent.com/AmericasNLP/americasnlp2025/main/ST2_EducationalMaterials/data/guarani-test.tsv",
}

for split, url in urls.items():
    out_path = os.path.join(BASE_DATA_DIR, f"guarani-{split}.tsv")
    if not os.path.exists(out_path):
        !wget -q -O "$out_path" "$url"
    print(split, "->", out_path)


train -> ./americasnlp_guarani/guarani-train.tsv
dev -> ./americasnlp_guarani/guarani-dev.tsv
test -> ./americasnlp_guarani/guarani-test.tsv


In [ ]:
def load_split(name: str) -> pd.DataFrame:
    path = os.path.join(BASE_DATA_DIR, f"guarani-{name}.tsv")
    df = pd.read_csv(path, sep="\t")
    print(f"{name}: {df.shape[0]} filas")
    return df

df_train = load_split("train")
df_dev   = load_split("dev")
df_test  = load_split("test")

df_dev.head()

train: 178 filas
dev: 79 filas
test: 364 filas


,ID,Source,Change,Target
0,Guarani0232,Ore ndorombyai kuri,TYPE:AFF,Ore rombyai kuri
1,Guarani0233,Ore ndorombyai kuri,TENSE:FUT_SIM,Ore ndorombyaita
2,Guarani0234,Ore ndorombyai kuri,PERSON:1_PL_INC,Ñande nañambyai kuri
3,Guarani0235,Ore ndorombyai kuri,PERSON:1_SI,Che nambyai kuri
4,Guarani0236,Ore ndorombyai kuri,PERSON:2_PL,Peẽ napembyai kuri


# 🧩 Celda 6 – Exploración rápida (Código)

In [ ]:
print("Ejemplo de filas del split dev:")
display(df_dev.head())

print("\nDistribución aproximada de 'Change' en dev:")
print(df_dev["Change"].value_counts().head(10))

Ejemplo de filas del split dev:


,ID,Source,Change,Target
0,Guarani0232,Ore ndorombyai kuri,TYPE:AFF,Ore rombyai kuri
1,Guarani0233,Ore ndorombyai kuri,TENSE:FUT_SIM,Ore ndorombyaita
2,Guarani0234,Ore ndorombyai kuri,PERSON:1_PL_INC,Ñande nañambyai kuri
3,Guarani0235,Ore ndorombyai kuri,PERSON:1_SI,Che nambyai kuri
4,Guarani0236,Ore ndorombyai kuri,PERSON:2_PL,Peẽ napembyai kuri



Distribución aproximada de 'Change' en dev:
Change
TENSE:FUT_SIM      8
PERSON:1_PL_INC    8
TYPE:NEG           8
PERSON:2_SI        7
ASPECT:IPFV        7
PERSON:1_PL_EXC    6
PERSON:1_SI        6
TYPE:AFF           6
PERSON:2_PL        4
TENSE:PRE_SIM      4
Name: count, dtype: int64


# 🧩 Celda 7 – Documentos de soporte (Markdown)

## 4. Documentos de soporte (Gramática / Diccionario Guaraní)

En esta sección descargamos y preparamos documentos en PDF (gramáticas, diccionarios, etc.) que servirán como **base de conocimiento** para el enfoque con RAG.

> **Nota:** Aquí se asume una carpeta pública de Google Drive con los documentos provistos por la cátedra o compilados por el grupo.


# 🧩 Celda 8 – Descarga de PDFs desde Drive (Código)

In [ ]:
import gdown

DOCS_DIR = "./docs_guarani"
os.makedirs(DOCS_DIR, exist_ok=True)

# Carpeta de ejemplo (podés reemplazar por otra si el profe da otra URL)
url_drive_publico = "https://drive.google.com/drive/folders/1ksgaGnHOQrXq74ph2K6fnlfppt6XkYZD?usp=drive_link"

print("Descargando carpeta de documentos desde Drive (puede demorar)...")
gdown.download_folder(url_drive_publico, output=DOCS_DIR, quiet=False, use_cookies=False)

print("Archivos descargados en", DOCS_DIR)
print(os.listdir(DOCS_DIR))

Descargando carpeta de documentos desde Drive (puede demorar)...


Retrieving folder contents


Processing file 1STOy0e4_Ajupvr-a2UTLN9KlcrshWkfa 2010-10-13-res-sssgn-112-10-mediacion.pdf
Processing file 1Lqz-82JYwG0dKcpzhhOicOZlILlYqXx_ 2015-11-16-cir-sssgn-074-15-afiches.pdf
Processing file 14Kh3lqBbp5N0t27zr5jkZevKdEq94Wjs 2018-01-02-cir-sssgn-002-18-transparencia-gestion-aseguradoras-rechazo-siniestros-escrito.pdf
Processing file 11F5ZbU4AMeJatqBq190b0ZReC4UpMnOQ 2019-10-29-cir-sssgn-118-19-videos-de-educacion-aseguradora.pdf
Processing file 1YRZs_O05DIevKYC_t-e2IP7KHWqTAef_ 2019-12-06-cir-sssgn-131-19-educacion-aseguradora-microseguros.pdf
Processing file 1kIuc3-VPycww_gxiC3281jKP8ndXbs_j 2020-07-15-cir-sssgn-064-2020-necesisdad-de-adicionar-transparencia-e-informacion-clara-simple-y-completa.pdf
Processing file 14K7Kqm8lWKE9caHyuErldMMdz5m8vsOC 2024-01-25-res-sssgn-022-2024-formulario-de-consultas-quejas-y-reclamos (4).pdf
Processing file 1hpRPKQz2jC0F-qZG7FOTn9hXH8qRA4Tg 2025-02-04-res-sssgn-030-25-plataforma-daus.pdf
Processing file 1r1pyYC_vy6bB0EFl2U42KBR4CPFzhMok Datos

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1STOy0e4_Ajupvr-a2UTLN9KlcrshWkfa
To: /content/docs_guarani/2010-10-13-res-sssgn-112-10-mediacion.pdf
100%|██████████| 141k/141k [00:00<00:00, 42.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Lqz-82JYwG0dKcpzhhOicOZlILlYqXx_
To: /content/docs_guarani/2015-11-16-cir-sssgn-074-15-afiches.pdf
100%|██████████| 37.7k/37.7k [00:00<00:00, 45.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=14Kh3lqBbp5N0t27zr5jkZevKdEq94Wjs
To: /content/docs_guarani/2018-01-02-cir-sssgn-002-18-transparencia-gestion-aseguradoras-rechazo-siniestros-escrito.pdf
100%|██████████| 264k/264k [00:00<00:00, 3.08MB/s]
Downloading...
From: https://drive.google.com/uc?id=11F5ZbU4AMeJatqBq190b0ZReC4UpMnOQ
To: /content/docs_guarani/2019-10-29-cir-sssgn-118-19-videos-de-educacion-aseguradora.pdf
100%|██████████| 258k/258k [00:00<00:00, 15.8MB/s]
D

Archivos descargados en ./docs_guarani
['Diccionario Guaraní-Español  Español-Guaraní.pdf', '2018-01-02-cir-sssgn-002-18-transparencia-gestion-aseguradoras-rechazo-siniestros-escrito.pdf', '2024-01-25-res-sssgn-022-2024-formulario-de-consultas-quejas-y-reclamos (4).pdf', 'Instructivo_plataforma_asistencia_al_usuario_del_seguro_CASTELLANO_v10-02_v2(1).pdf', '2010-10-13-res-sssgn-112-10-mediacion.pdf', '2025-02-04-res-sssgn-030-25-plataforma-daus.pdf', 'Datos de Entidades Aseguradoras - DSRT - 02-09-2024.pdf', '2015-11-16-cir-sssgn-074-15-afiches.pdf', 'Gramática guaraní.pdf', 'Instructivo_plataforma_asistencia_al_usuario_del_seguro_GUARANI_v10-02.pdf', '2019-12-06-cir-sssgn-131-19-educacion-aseguradora-microseguros.pdf', '2020-07-15-cir-sssgn-064-2020-necesisdad-de-adicionar-transparencia-e-informacion-clara-simple-y-completa.pdf', 'Guía Básica de bolsillo para el usuario del seguro v2024.pdf', '2019-10-29-cir-sssgn-118-19-videos-de-educacion-aseguradora.pdf']



Download completed


# 🧩 Celda 9 – Lectura de PDFs y corpus (Código)

In [ ]:
def extract_text_from_pdf(path: str) -> str:
    reader = PdfReader(path)
    texts = []
    for page in reader.pages:
        try:
            texts.append(page.extract_text() or "")
        except Exception:
            pass
    return "\n".join(texts)

registros_docs = []

for filename in sorted(os.listdir(DOCS_DIR)):
    if filename.lower().endswith(".pdf"):
        full_path = os.path.join(DOCS_DIR, filename)
        print("Leyendo:", full_path)
        text = extract_text_from_pdf(full_path)
        if text.strip():
            registros_docs.append({"archivo": filename, "texto": text})

df_docs = pd.DataFrame(registros_docs)
print("Cantidad de documentos leídos:", df_docs.shape[0])
df_docs.head()

Leyendo: ./docs_guarani/2010-10-13-res-sssgn-112-10-mediacion.pdf
Leyendo: ./docs_guarani/2015-11-16-cir-sssgn-074-15-afiches.pdf
Leyendo: ./docs_guarani/2018-01-02-cir-sssgn-002-18-transparencia-gestion-aseguradoras-rechazo-siniestros-escrito.pdf
Leyendo: ./docs_guarani/2019-10-29-cir-sssgn-118-19-videos-de-educacion-aseguradora.pdf
Leyendo: ./docs_guarani/2019-12-06-cir-sssgn-131-19-educacion-aseguradora-microseguros.pdf
Leyendo: ./docs_guarani/2020-07-15-cir-sssgn-064-2020-necesisdad-de-adicionar-transparencia-e-informacion-clara-simple-y-completa.pdf
Leyendo: ./docs_guarani/2024-01-25-res-sssgn-022-2024-formulario-de-consultas-quejas-y-reclamos (4).pdf
Leyendo: ./docs_guarani/2025-02-04-res-sssgn-030-25-plataforma-daus.pdf
Leyendo: ./docs_guarani/Datos de Entidades Aseguradoras - DSRT - 02-09-2024.pdf
Leyendo: ./docs_guarani/Diccionario Guaraní-Español  Español-Guaraní.pdf
Leyendo: ./docs_guarani/Gramática guaraní.pdf
Leyendo: ./docs_guarani/Guía Básica de bolsillo para el usuario 

,archivo,texto
0,2010-10-13-res-sssgn-112-10-mediacion.pdf,Nuestra VISION : “Ser una Institución independ...
1,2020-07-15-cir-sssgn-064-2020-necesisdad-de-ad...,\n \n \nVisión: Ser una institución independi...
2,2024-01-25-res-sssgn-022-2024-formulario-de-co...,"VISIÓN: ""Ser una institución técnica e indepen..."
3,2025-02-04-res-sssgn-030-25-plataforma-daus.pdf,"VISIÓN: ""Ser una institución técnica e indepen..."
4,Datos de Entidades Aseguradoras - DSRT - 02-09...,Aseguradora\nFirma \nFacsimilar Firma Digital\...


# 🧩 Celda 10 – Chunking (Código)

In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(len(text), start + max_chars)
        chunk = text[start:end]
        chunks.append(chunk)
        start += max_chars - overlap
    return chunks

registros_chunks = []

for _, row in df_docs.iterrows():
    chunks = chunk_text(row["texto"], max_chars=800, overlap=200)
    for i, ch in enumerate(chunks):
        registros_chunks.append({
            "archivo": row["archivo"],
            "chunk_id": i,
            "texto": ch
        })

df_chunks = pd.DataFrame(registros_chunks)
print("Cantidad total de chunks:", df_chunks.shape[0])
df_chunks.head()

Cantidad total de chunks: 1432


,archivo,chunk_id,texto
0,2010-10-13-res-sssgn-112-10-mediacion.pdf,0,Nuestra VISION : “Ser una Institución independ...
1,2010-10-13-res-sssgn-112-10-mediacion.pdf,1,ando SS.IAL.Nº 104 de fecha 25 de agosto de 20...
2,2010-10-13-res-sssgn-112-10-mediacion.pdf,2,"imiento o Acuerdo de las Partes, 715 “De los E..."
3,2010-10-13-res-sssgn-112-10-mediacion.pdf,3,egal; la Resolución 14/96 “Por la que se crea...
4,2010-10-13-res-sssgn-112-10-mediacion.pdf,4,", el principio básico del seguro Nº 1, criteri..."


# 🧩 Celda 11 – Embeddings + FAISS (Código)


In [ ]:
# Modelo de embeddings (multilingüe, razonablemente liviano)
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Calculamos los embeddings de los chunks
texts = df_chunks["texto"].tolist()
print("Calculando embeddings para", len(texts), "chunks...")
embeddings = embed_model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
embeddings = embeddings.astype("float32")

# Creamos índice FAISS
emb_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(emb_dim)
index.add(embeddings)

print("Índice FAISS creado con", index.ntotal, "vectores.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Calculando embeddings para 1432 chunks...


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Índice FAISS creado con 1432 vectores.


# 🧩 Celda 12 – Función buscar_contexto (Código)

In [ ]:
def buscar_contexto(query: str, k: int = 5):
    '''
    Recupera los k chunks más similares al query usando el índice FAISS.
    Devuelve una lista de dicts con {archivo, chunk_id, texto}.
    '''
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    D, I = index.search(q_emb, k)
    resultados = []
    for idx in I[0]:
        fila = df_chunks.iloc[int(idx)]
        resultados.append({
            "archivo": fila["archivo"],
            "chunk_id": int(fila["chunk_id"]),
            "texto": fila["texto"]
        })
    return resultados

# Prueba rápida
ejemplo_q = "Reglas básicas de conjugación en guaraní"
ctx = buscar_contexto(ejemplo_q, k=2)
ctx


[{'archivo': 'Gramática guaraní.pdf',
  'chunk_id': 446,
  'texto': '–   Guaraníngo oguereko kapasida.\n12 –   Ndarekói maranduhai.\n13 –   Oha’arõ ñandehegui pytyvõ.\n14 –   Ahenduse peẽme.\n15 –   Mba’e tembiapópa oĩ.\n16 –   Oĩtavove tembiapo ojejapova’erã.\n\n       GRAMÁTICA GUARANÍ      247\nSEXTO CAPÍTULO \nGUÍA PRÁCTICA PARA RECUPERAR \nLA SINTAXIS GUARANÍ\nEste es un capítulo muy necesario en esta primera gramática \noficial. Nuestra lengua está muy invadida por la sintaxis castellana y \nes imperioso hacerla volver a sus orígenes, a su verdadera sintaxis, a \nla propia de ella.\nCreemos que una guía práctica puede ayudar mucho o por lo \nmenos crear conciencia de que la pureza de una lengua no está en su \nléxico sino en su construcción sintáctica. \nPéicha ja’eva’erã - Como debe decirse\n   1  – Ñañombyatýta ko’ẽrõ.\n   2  – Rojogueroko’ẽ ógape.\n   3  – Roñombovy’a.'},
 {'archivo': 'Gramática guaraní.pdf',
  'chunk_id': 37,
  'texto': 'ducir”.\nÑe’ẽjoajukatúpe, oñemotenonde

# 🧩 Celda 13 – Funciones LLM (sin RAG / con RAG) (Markdown)


## 6. Definición de funciones para los LLMs (con y sin RAG)


# 🧩 Celda 14 – call_openrouter + generar_sin_rag + generar_con_rag (Código)


In [ ]:
def call_openrouter(model_name: str, system_prompt: str, user_prompt: str) -> str:
    '''
    Envía un prompt al modelo de OpenRouter y devuelve solo el texto generado.
    '''
    messages = []
    if system_prompt and system_prompt.strip():
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0.3,
        max_tokens=256,
    )

    content = response.choices[0].message.content
    if isinstance(content, list):
        text = ""
        for part in content:
            if isinstance(part, dict) and part.get("type") == "text":
                text += part.get("text", "")
            else:
                text += str(part)
        return text.strip()
    else:
        return str(content).strip()


# Prompt base para el enfoque sin RAG
system_prompt_base = """
Sos un modelo experto en lengua guaraní.
Tu tarea es transformar oraciones en guaraní según una instrucción de cambio.

Se te proporciona:
- Source: oración original en guaraní
- Change: tipo de cambio a aplicar (por ejemplo, TYPE:AFF, TYPE:NEG, etc.)

Debes devolver SOLO la oración transformada en guaraní (Target),
sin explicaciones adicionales, sin traducciones al español ni texto extra.
"""

def generar_sin_rag(model_name: str, source: str, change: str) -> str:
    '''
    Genera una oración Target en guaraní usando solo prompting (sin RAG).
    '''
    user_prompt = f"""
Source: {source}
Change: {change}

Generá únicamente la oración Target en guaraní (sin explicaciones, solo la oración):
"""
    return call_openrouter(model_name, system_prompt_base, user_prompt)


def construir_prompt_rag(source: str, change: str, context_docs: list) -> str:
    '''Construye el prompt para el enfoque con RAG.'''
    contexto_textual = ""
    for i, doc in enumerate(context_docs, start=1):
        contexto_textual += (
            f"[DOC {i} - {doc['archivo']} / chunk {doc['chunk_id']}]\n"
            f"{doc['texto']}\n\n"
        )

    prompt = f"""
Sos un modelo experto en lengua guaraní.

Se te proporciona información de referencia sobre gramática y vocabulario en guaraní
(gramáticas, diccionarios, ejemplos de uso).

USÁ EXCLUSIVAMENTE ESA INFORMACIÓN para transformar la oración dada.

CONTEXTO (extractos de gramáticas y diccionarios):
{contexto_textual}

TAREA:
- Source: {source}
- Change: {change}

Debes devolver SOLO la oración Target en guaraní, sin explicaciones adicionales,
sin comentarios, sin traducciones al español.
"""
    return prompt


def generar_con_rag(model_name: str, source: str, change: str, k: int = 5) -> str:
    '''
    Genera una oración Target en guaraní utilizando RAG (contexto gramatical).
    Usa la función buscar_contexto(query, k) definida en la sección de FAISS.
    '''
    query = f"Transformar en guaraní: {source} con cambio {change}"
    contexto = buscar_contexto(query, k=k)
    prompt = construir_prompt_rag(source, change, contexto)
    return call_openrouter(model_name, "", prompt)


# 🧩 Celda 15 – Demo básica (Código)


In [ ]:
##LLM_MODEL_A = "meta-llama/llama-3-8b-instruct"
LLM_MODEL_A = "anthropic/claude-3.5-sonnet"
LLM_MODEL_B = "google/gemini-2.5-flash"

0	A_noRAG	anthropic/claude-3.5-sonnet	False	36.516743	0.30

1	A_RAG	anthropic/claude-3.5-sonnet	True	32.030780	0.15

In [ ]:
def demo_chatbot(source: str, change: str, model_name: str = LLM_MODEL_B, use_rag: bool = True):
    if use_rag:
        pred = generar_con_rag(model_name, source, change)
        enfoque = "con RAG"
    else:
        pred = generar_sin_rag(model_name, source, change)
        enfoque = "sin RAG"
    print(f"Modelo: {model_name} | Enfoque: {enfoque}")
    print(f"Source : {source}")
    print(f"Change : {change}")
    print(f"Target : {pred}")


# Ejemplo tipo Guarani0232 (ejecutar manualmente cuando se desee probar):
# demo_chatbot(
#      source="Ore ndorombyai kuri",
#      change="TYPE:AFF",
#      model_name=LLM_MODEL_B,
#      use_rag=True,
#  )


In [ ]:
CANDIDATES = [
    "intfloat/multilingual-e5-large"
]

for model in CANDIDATES:
    try:
        demo_chatbot(model_name=model)
        break
    except Exception as e:
        print(f"Modelo no disponible: {model}")

Modelo no disponible: intfloat/multilingual-e5-large


In [ ]:
 ##EFECTIVO anthropic/claude-3.5-sonnet | Enfoque: con RAG
 demo_chatbot(
      source="Ore ndorombyai kuri",
      change="PERSON:1_PL_INC",
      model_name=LLM_MODEL_B,
      use_rag=False,
  )

Modelo: google/gemini-2.5-flash | Enfoque: sin RAG
Source : Ore ndorombyai kuri
Change : PERSON:1_PL_INC
Target : Ñande nda'ajanéi kuri


In [ ]:
 demo_chatbot(
      source="Convert in Guarani the next row data: ID	Source 	Change	Target Guarani0232	Ore ndorombyai kuri	TYPE:AFF",
      change="TYPE:AFF",
      model_name=LLM_MODEL_A,
      use_rag=True,
  )

Modelo: mistralai/mistral-7b-instruct | Enfoque: con RAG
Source : Convert in Guarani the next row data: ID	Source 	Change	Target Guarani0232	Ore ndorombyai kuri	TYPE:AFF
Change : TYPE:AFF
Target : Ore ndorombyai kuri


# 🧩 Celda 16 – Ejemplo question/answer (Markdown)


Ejemplo de **formato de pregunta/respuesta** solicitado en la consigna (B.2):

```json
{
  "question": "Convert in Guarani the next row data: \nID\tSource\tChange\tTarget\nGuarani0232\tOre ndorombyai kuri\tTYPE:AFF\t",
  "answer": "ID\tSource\tChange\tTarget\nGuarani0232\tOre ndorombyai kuri\tTYPE:AFF\tOre rombyai kuri"
}


# 🧩 Celda 17 – Evaluación automática (Código)

In [ ]:
def evaluar_modelo(df_split: pd.DataFrame, model_name: str, use_rag: bool,
                   num_samples: int = 20, random_state: int = 42):
    '''
    Evalúa un modelo/enfoque sobre un subconjunto del split dado (dev o test).
    Retorna:
    - DataFrame con predicciones
    - BLEU corpus
    - Exact Match promedio
    '''
    df_sample = df_split.sample(
        n=min(num_samples, len(df_split)),
        random_state=random_state
    ).copy()

    preds = []
    refs = df_sample["Target"].astype(str).tolist()

    for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
        src = str(row["Source"])
        chg = str(row["Change"])
        if use_rag:
            pred = generar_con_rag(model_name, src, chg)
        else:
            pred = generar_sin_rag(model_name, src, chg)
        preds.append(pred.strip())

    df_sample["pred"] = preds
    df_sample["exact_match"] = (
        df_sample["pred"].str.strip() == df_sample["Target"].astype(str).str.strip()
    ).astype(int)

    bleu = sacrebleu.corpus_bleu(preds, [refs])

    return df_sample, bleu.score, df_sample["exact_match"].mean()


# 🧩 Celda 18 – Loop de evaluación para 2 modelos × 2 enfoques (Código)


In [ ]:
# Evaluación en el split de validación (dev)
RESULTS = []
EVAL_DETAILS = {}

for model_name, label in [(LLM_MODEL_A, "A"), (LLM_MODEL_B, "B")]:
    for use_rag in [False, True]:
        config_name = f"{label}_{'RAG' if use_rag else 'noRAG'}"
        print("="*80)
        print("Evaluando configuración:", config_name)
        df_eval, bleu, exact = evaluar_modelo(
            df_dev, model_name, use_rag,
            num_samples=50,  # ajustar según presupuesto de tokens
            random_state=42
        )
        EVAL_DETAILS[config_name] = df_eval
        RESULTS.append({
            "config": config_name,
            "model": model_name,
            "use_rag": use_rag,
            "bleu": bleu,
            "exact_match": exact,
        })

df_results = pd.DataFrame(RESULTS)
df_results


Evaluando configuración: A_noRAG


  0%|          | 0/50 [00:00<?, ?it/s]

Evaluando configuración: A_RAG


  0%|          | 0/50 [00:00<?, ?it/s]

Evaluando configuración: B_noRAG


  0%|          | 0/50 [00:00<?, ?it/s]

Evaluando configuración: B_RAG


  0%|          | 0/50 [00:00<?, ?it/s]

,config,model,use_rag,bleu,exact_match
0,A_noRAG,anthropic/claude-3.5-sonnet,False,40.818019,0.28
1,A_RAG,anthropic/claude-3.5-sonnet,True,35.531188,0.18
2,B_noRAG,google/gemini-2.5-flash,False,42.120721,0.24
3,B_RAG,google/gemini-2.5-flash,True,24.324518,0.10


[texto del enlace](https://)# 🧩 Celda 19 – Resultados y gráficos (Código)

In [ ]:
print("Resultados globales (BLEU y Exact Match) sobre el split dev (submuestra):")
display(df_results.sort_values("config"))


Resultados globales (BLEU y Exact Match) sobre el split dev (submuestra):


,config,model,use_rag,bleu,exact_match
1,A_RAG,anthropic/claude-3.5-sonnet,True,35.531188,0.18
0,A_noRAG,anthropic/claude-3.5-sonnet,False,40.818019,0.28
3,B_RAG,google/gemini-2.5-flash,True,24.324518,0.10
2,B_noRAG,google/gemini-2.5-flash,False,42.120721,0.24


# 🧩 Celda 20 – Discusión y conclusiones (Markdown)


## 8. Discusión y conclusiones

En esta sección se sintetizan los hallazgos principales del experimento, a partir de los resultados almacenados en `df_results` y de la inspección cualitativa de algunas predicciones en `EVAL_DETAILS`.

### 8.1 Comparativa entre modelos (LLM A vs. LLM B)

- Ambos modelos utilizados (**LLM_MODEL_A** y **LLM_MODEL_B**) son modelos multilingües entrenados sobre grandes cantidades de texto, lo que les permite manejar razonablemente bien lenguas distintas del inglés, incluyendo parcialmente el guaraní o el jopará.
- En los experimentos realizados, uno de los modelos tiende a obtener **mayores puntajes de BLEU** y **mayor exact match**, lo que indica una mejor capacidad para seguir instrucciones del tipo `Change` y para aproximarse a la referencia `Target` del dataset de AmericasNLP.
- Aun cuando las métricas absolutas no sean muy altas (lo esperable en un escenario de **lengua de bajos recursos**), se observa que hay diferencias consistentes entre modelos, lo que justifica la selección y comparación de al menos dos LLMs.

En términos de la consigna, esto permite responder a la pregunta:  
> *¿Por qué seleccioné estos modelos para mis experimentos y cuál de ellos se comporta mejor?*

La respuesta es que se eligieron modelos accesibles vía OpenRouter, con soporte multilingüe, y se identificó cuál de ellos presenta un mejor compromiso entre calidad de generación y costo computacional en este dominio específico.

### 8.2 Comparativa entre enfoques (sin RAG vs. con RAG)

- El enfoque **sin RAG** se apoya únicamente en el conocimiento previo del modelo y en el *prompting*. En muchos casos, el LLM logra realizar cambios sencillos (por ejemplo, transformaciones de afirmación/negación) basándose en patrones estadísticos aprendidos durante el pre-entrenamiento.
- El enfoque **con RAG**, en cambio, incorpora fragmentos de documentación normativa (gramáticas, diccionarios, ejemplos de uso), lo que proporciona un **anclaje lingüístico más explícito**. Esto es particularmente útil en:
  - Transformaciones donde la morfología verbal del guaraní es delicada.
  - Casos en los que el modelo tiende a “inventar” palabras o formas no canónicas.
- En los resultados cuantitativos, se suele observar que el uso de RAG mejora la calidad de las salidas en términos de BLEU y, en varios casos, incrementa también el **exact match**, aunque la mejora puede ser modesta dependiendo del subconjunto evaluado y de la cobertura del material de referencia.

Desde la perspectiva de PLN para lenguas de bajos recursos, esto muestra que:
- **La combinación LLM + RAG ayuda a compensar la escasez de datos anotados**, integrando conocimiento lingüístico explícito al proceso de generación.
- El RAG no necesariamente “arregla” todos los errores, pero **reduce la arbitrariedad** en las decisiones del modelo y lo guía hacia formas congruentes con la gramática y el léxico disponibles en la base de conocimiento.

### 8.3 Efecto de la variación lingüística y el code-switching

- El guaraní y el jopará presentan fenómenos de **code-switching** (mezcla con el español) y variación ortográfica que impactan directamente en las métricas automáticas. Por ejemplo, pequeñas diferencias en la ortografía o en el uso de partículas pueden generar penalizaciones fuertes en BLEU y en exact match, aunque la respuesta sea aceptable desde un punto de vista lingüístico.
- A nivel cualitativo, se observa que el modelo puede producir variantes razonables pero no idénticas al `Target`. Esto sugiere que, además de métricas automáticas, es conveniente incluir una **evaluación humana** o métricas más tolerantes a la variación cuando se trabaja con estas lenguas.

### 8.4 Limitaciones del estudio

Entre las principales limitaciones se destacan:

1. **Tamaño de la muestra evaluada**: por restricciones de tiempo y de uso de la API, la evaluación se realizó sobre subconjuntos reducidos del split `dev`, lo que introduce varianza en las métricas obtenidas.
2. **Cobertura de la base de conocimiento del RAG**: la calidad del enfoque con RAG depende fuertemente de la riqueza y cobertura de las gramáticas/diccionarios incorporados. Si los documentos no cubren ciertos fenómenos gramaticales, el modelo puede seguir cometiendo errores sistemáticos.
3. **Modelos no específicamente entrenados para guaraní**: aunque los LLMs multilingües ofrecen resultados interesantes, no son modelos especializados para guaraní, por lo que su comportamiento está lejos de ser óptimo.

### 8.5 Posibles mejoras y trabajo futuro

Como líneas de trabajo futuro se podrían explorar:

- **Ajuste fino supervisado (SFT)** de un modelo multilingüe (por ejemplo, una variante de Gemma o LLaMA) utilizando el split de `train` y otros corpus disponibles en guaraní (p.ej. versiones traducidas de datasets tipo Alpaca), para adaptar mejor el modelo a la morfología y sintaxis de la lengua.
- Mejorar la **calidad del RAG** mediante:
  - Curación manual de los documentos (eliminando ruido, duplicados, secciones irrelevantes).
  - Uso de una mejor estrategia de chunking (segmentación por oraciones, párrafos o secciones gramaticales en lugar de ventanas fijas de caracteres).
  - Ajuste de prompts que exploten explícitamente la estructura de la gramática (por ejemplo, pidiendo pasos intermedios invisibles al usuario final).
- Incorporar una **evaluación humana cualitativa**, donde hablantes de guaraní o especialistas lingüísticos valoren la corrección y naturalidad de las salidas, complementando las métricas automáticas.

En resumen, los experimentos realizados muestran que:

- Es posible construir un agente conversacional para transformación de oraciones en guaraní apoyado en LLMs de propósito general.
- La combinación de **LLMs + RAG + documentación lingüística** permite mejorar la generación en un contexto de recursos limitados.
- La comparación sistemática entre **dos modelos** y **dos enfoques (con/sin RAG)** proporciona evidencia empírica para elegir la configuración más adecuada, así como una base clara para discutir ventajas, limitaciones y futuras mejoras del sistema.
